In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [3]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [4]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [5]:
# d3 = [273, 372, 221]

In [6]:
ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']

In [7]:
pbar = tqdm()

0it [00:00, ?it/s]

In [8]:
not_ran = [380, 343, 360, 159, 346]

In [ ]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in [300]:
    not_ran_operators = []
    # if agency in ran_operators:
    #     print(f'already ran: {agency}')
    #     continue
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        # rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
        # rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_02_08_pm_peak.html')
        rt_day.stop_segment_speed_view.to_parquet(f'./segment_speed_views/{agency}_02_08_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        delay_to_parquet.to_parquet(f'./endpoint_delay_views/{agency}_02_08_pm_peak.parquet') ## early step towards scaling
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 300...
found parquet
found parquet
found parquet
found parquet
found_parquet
less than 1km of data


In [ ]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')

In [4]:
gpd.read_parquet('./segment_speed_views/170_02_08_pm_peak.parquet')

,stop_id,stop_name,geometry,shape_id,shape_meters,stop_sequence,route_id,direction_id,seconds_from_last,last_loc,meters_from_last,speed_from_last,speed_mph,n_trips,avg_mph,_20p_mph,trips_per_hour
0,4013,LONG BEACH BLVD & 4TH SE,"LINESTRING (167757.218 -470026.899, 167757.055...",1720090,578.149080,3,172,1,23.0,446.220653,131.928426,5.736019,12.831473,4,11.789760,10.189541,1.0
1,4013,LONG BEACH BLVD & 4TH SE,"LINESTRING (167757.218 -470026.899, 167757.055...",1720090,578.149080,3,172,1,25.0,446.220653,131.928426,5.277137,11.804956,4,11.789760,10.189541,1.0
2,4013,LONG BEACH BLVD & 4TH SE,"LINESTRING (167757.218 -470026.899, 167757.055...",1720090,578.149080,3,172,1,38.0,446.220653,131.928426,3.471801,7.766418,4,11.789760,10.189541,1.0
3,4013,LONG BEACH BLVD & 4TH SE,"LINESTRING (167757.218 -470026.899, 167757.055...",1720090,578.149080,3,172,1,20.0,446.220653,131.928426,6.596421,14.756194,4,11.789760,10.189541,1.0
4,0617,SEVENTH & L.B. BLVD. NW,"LINESTRING (167754.327 -469895.008, 167754.033...",1720090,1045.747670,4,172,1,65.0,578.149080,467.598591,7.193824,16.092585,8,8.179137,6.097478,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,2002,2660 Del Amo Blvd S,"LINESTRING (164385.844 -461642.155, 164385.886...",10075,7575.224464,15,1,0,77.0,6721.481001,853.743464,11.087577,24.802911,6,20.973872,17.521322,1.5
51,2002,2660 Del Amo Blvd S,"LINESTRING (164385.844 -461642.155, 164385.886...",10075,7575.224464,15,1,0,73.0,6721.481001,853.743464,11.695116,26.161974,6,20.973872,17.521322,1.5
52,2002,2660 Del Amo Blvd S,"LINESTRING (164385.844 -461642.155, 164385.886...",10075,7575.224464,15,1,0,82.0,6721.481001,853.743464,10.411506,23.290538,6,20.973872,17.521322,1.5
53,2002,2660 Del Amo Blvd S,"LINESTRING (164385.844 -461642.155, 164385.886...",10075,7575.224464,15,1,0,101.0,6721.481001,853.743464,8.452906,18.909150,6,20.973872,17.521322,1.5
